In [10]:
# documentation scipy.stats.norm: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

In [1]:
from scipy.stats import norm
import numpy as np

In [2]:
# EMSR sample inputs
r1 = [300, 200, 100, 50, 25]  # prices
mu1 = [10, 15, 25, 50, 500]  # means
sigma1 = [2, 2, 3, 3, 5]  # sds
C1 = 160  # capacity

r2 = [500, 400, 200, 100, 50, 25]  # ordered prices per class  
mu2 = [4, 8, 15, 25, 50, 500]  # ordered class demand means 
sigma2 = [2, 2, 4, 10, 20, 50]  # ordered class standard deviations
C2 = 200  # capacity

In [3]:
def generate_indexes_k_l(array):
    length = len(array)
    # Initialize an empty array to hold all sub-arrays
    nested_array = []

    # Iterate from 1 to the specified length
    for n in range(1, length):
        sub_array = []
        # Create sub-array with elements [n, i] where i ranges from 0 to n-1
        for i in range(n):
            sub_array.append([n, i])
        # Append each sub-array to the main array
        nested_array.append(sub_array)

    return nested_array

def calculate_emrs_a(r, mu, sigma, C):
    indexes = generate_indexes_k_l(r)
    protection_limits = []
    G_k = 0
    group_size = 1
    count = 0

    # Calculate protection limits
    for sub_array in indexes:
        for element in sub_array:
            k = element[0]
            l = element[1]
            if l == 0:
                G_k_l = np.round(norm(mu[0], sigma[0]).ppf(1 - r[k] / r[0]), 0)
            else:
                G_k_l = np.round(norm(mu[l], sigma[l]).ppf(1 - r[k] / r[l]), 0)

            # Add the current value to the running sum
            G_k += G_k_l
            count += 1

            # Check if we've reached the group size
            if count == group_size:
                protection_limits.append(G_k)
                G_k = 0
                group_size += 1
                count = 0

    # Add the constant C to the final group
    G_k += C
    protection_limits.append(G_k)
    
    # Calculate booking limits
    booking_limits = [C]
    for i in range(len(protection_limits)-1):
        booking_limits.append(C - protection_limits[i]) 


    return protection_limits, booking_limits

# Example usage
indexes = generate_indexes_k_l(r2)
print(indexes)

[[[1, 0]], [[2, 0], [2, 1]], [[3, 0], [3, 1], [3, 2]], [[4, 0], [4, 1], [4, 2], [4, 3]], [[5, 0], [5, 1], [5, 2], [5, 3], [5, 4]]]


In [4]:
# Task 5
protection_limits, booking_limits = calculate_emrs_a(r1, mu1, sigma1, C1)
print(f"Protection Limits: {protection_limits}")
print(f"Booking Limits: {booking_limits}")

Protection Limits: [9.0, 26.0, 53.0, 107.0, 160]
Booking Limits: [160, 151.0, 134.0, 107.0, 53.0]


In [5]:
def generate_indexes_k(array):
    n = len(array)  # Get the length of the input array
    result = []  # Initialize an empty list to hold the result
    for i in range(n):
        # Create a list from 0 to i
        current_list = list(range(i + 1))
        result.append(current_list)  # Append the current list to the result
    return result  # Return the final result as an array

def calculate_emrs_b(r, mu, sigma, C):
    # Generate indexes using the provided price array (r)
    indexes = generate_indexes_k(r)

    G = []  # Initialize list for protection limits
    B = [C]  # Initialize B with capacity C

    for index in indexes[:-1]:        
        mu_i = sum(mu[j] for j in index)  # Cumulative demand mean
        sigma_i = np.sqrt(sum(sigma[j]**2 for j in index))  # Cumulative standard deviation

        # Calculate r_i
        Zähler = sum(r[j] * mu[j] for j in index)
        Nenner = sum(mu[j] for j in index)
        r_i = Zähler / Nenner

        # Calculate G_i using the percent point function (PPF)
        probability_i = 1 - (r[index[-1]+1] / r_i)
        G_i = np.round(norm(mu_i, sigma_i).ppf(probability_i), 0)
        G.append(G_i)
    G.append(C)  # Add the capacity to the protection limits

    # Calculate booking limits
    for i in range(len(G)-1):
        B.append(C - G[i])

    return G, B

    
# Example usage
output = generate_indexes_k(r2)
print(output)

[[0], [0, 1], [0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5]]


In [6]:
# Task 6
protection_limits, booking_limits = calculate_emrs_b(r2, mu2, sigma2, C2)
print(f"Protection Limits: {protection_limits}")
print(f"Booking Limits: {booking_limits}")

Protection Limits: [2.0, 12.0, 29.0, 60.0, 122.0, 200]
Booking Limits: [200, 198.0, 188.0, 171.0, 140.0, 78.0]
